In [1]:
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('/data/datasets/mimic3_18var/root/in-hospital-mortality/train_listfile.csv')

In [3]:
train_data

,stay,y_true
0,44856_episode1_timeseries.csv,0
1,4016_episode2_timeseries.csv,0
2,20671_episode1_timeseries.csv,0
3,72867_episode1_timeseries.csv,0
4,31031_episode1_timeseries.csv,1
...,...,...
14676,22128_episode1_timeseries.csv,0
14677,86158_episode2_timeseries.csv,1
14678,10577_episode1_timeseries.csv,0
14679,9798_episode1_timeseries.csv,0


In [4]:
sample_path = train_data['stay'][1]
data_dir = "/data/datasets/mimic3_18var/root/in-hospital-mortality/train/"

In [5]:
sample_path = data_dir + sample_path

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn import functional as F

In [7]:
df = pd.read_csv(sample_path)

In [8]:
print(df.columns)

Index(['Hours', 'CO2 (ETCO2, PCO2, etc.)', 'Capillary refill rate',
       'Diastolic blood pressure', 'Fraction inspired oxygen',
       'Glascow coma scale eye opening', 'Glascow coma scale motor response',
       'Glascow coma scale total', 'Glascow coma scale verbal response',
       'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure',
       'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure',
       'Temperature', 'Weight', 'pH'],
      dtype='object')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SinusoidalEmbedding(nn.Module):
    def __init__(self, embedding_dim):
        super(SinusoidalEmbedding, self).__init__()
        self.embedding_dim = embedding_dim

    def forward(self, input_hours):
        # Calculate sinusoidal embedding
        position = torch.arange(0, self.embedding_dim, dtype=torch.float32).unsqueeze(0)
        div_term = torch.exp(torch.arange(0, self.embedding_dim, 2, dtype=torch.float32) * -(np.log(10000.0) / self.embedding_dim))
        
        # Calculate sine and cosine terms
        sin_terms = torch.sin(input_hours.unsqueeze(-1) * div_term)
        cos_terms = torch.cos(input_hours.unsqueeze(-1) * div_term)

        # Interleave sine and cosine terms
        sinusoidal_embedding = torch.empty(input_hours.size(0), self.embedding_dim, dtype=torch.float32)
        sinusoidal_embedding[:, 0::2] = sin_terms
        sinusoidal_embedding[:, 1::2] = cos_terms
        return sinusoidal_embedding

# Example usage
embedding_dim = 64  # You can adjust this based on your requirements
sinusoidal_embedding = SinusoidalEmbedding(embedding_dim)

# Input in hours (assuming 24 hours in a day)
input_hours = torch.tensor([0, 6, 12, 18], dtype=torch.float32)
output_embedding = sinusoidal_embedding(input_hours)

print("Input hours:", input_hours)
print("Sinusoidal Embedding:")
print(output_embedding.shape)


In [ ]:
output_embedding[1]

In [ ]:
df['Glascow coma scale motor response'].unique()

In [ ]:
df['Glascow coma scale verbal response'].unique()

In [ ]:
id_name_dict = {}
# df.drop(labels=categorical_variables, axis=1, inplace=True)
for i in range(len(df.columns)):
    id_name_dict[i] = df.columns[i]
values = df.values

In [ ]:
name_id_dict = {v:k for k,v in id_name_dict.items()}

In [ ]:
categorical_variables = ['Glascow coma scale eye opening', 
                                 'Glascow coma scale motor response', 
                                 'Glascow coma scale verbal response']

In [ ]:
df.drop(labels=categorical_variables, axis=1, inplace=True)

In [ ]:
id_name_dict = {}

for i in range(len(df.columns)):
    id_name_dict[i] = df.columns[i]
values = df.values

In [ ]:
sample = [[],[],[],[]]
for i in range(values.shape[0]):
    time = values[i,0]
    for j in range(1, values.shape[1]):
        try :
            np.isnan(values[i][j])
        except:
            print(values[i][j])
        if np.isnan(values[i][j]) == False:
            sample[0].append(time)
            sample[1].append(values[i][j])
            sample[2].append(j)
            sample[3].append(id_name_dict[j])

In [ ]:
max(sample[2])

In [ ]:
categorical_variables = ['Glascow coma scale eye opening', 
                                 'Glascow coma scale motor response', 
                               'Glascow coma scale verbal response']
from tqdm import tqdm
cat_set_list = []
for cat in categorical_variables:
    cat_set = set()
    for sample_path in tqdm(train_data['stay']):
        sample_path = data_dir + sample_path
        df = pd.read_csv(sample_path)
        df.fillna(0, inplace=True)
        cat_set = cat_set.union(set(df[cat].unique()))
    cat_set_list.append(cat_set)
        

In [ ]:
cat_set_list[0]

In [ ]:
cat_set_list[1]

In [ ]:
cat_set_list[2]

In [ ]:
cat_set_list

In [ ]:
train_data

In [ ]:
data_dir

In [32]:
pd.set_option('future.no_silent_downcasting',True)
from tqdm import tqdm
def isNAN(x):
    return x!=x
def get_mean_var(data, data_dir):
    categorical_variables = ['Glascow coma scale eye opening', 
                                 'Glascow coma scale motor response', 
                                 'Glascow coma scale verbal response']
    sample_path = data_dir + data['stay'][0]
    id_name_dict = {}
    df = pd.read_csv(sample_path)
    df.drop(labels=categorical_variables, axis=1, inplace=True)
    for i in range(len(df.columns)):
        id_name_dict[i] = df.columns[i]
    variable_values = {k : [] for k in df.columns[1:]}
    for sample_path in tqdm(data['stay']):
        sample_path = data_dir+sample_path
        df = pd.read_csv(sample_path)
        values = df.values
        df.drop(labels=categorical_variables, axis=1, inplace=True)
        df.replace(['ERROR','no data','.','-'], np.nan, inplace=True)
        cols = df.columns[1:]
        # print(len(cols))
        df = df[cols]
        values = df.values
        # print(values.shape)
        # print(len(id_name_dict))
        for i in range(values.shape[0]):
            for j in range(values.shape[1]):
                
                if isNAN((values[i][j])) == False:
                    variable_values[id_name_dict[j+1]].append(float(values[i][j]))
    result_dict = {}
    for feature, values in variable_values.items():
        mean_value = np.mean(values)
        variance_value = np.var(values)
        result_dict[feature] = {'mean': mean_value, 'variance': variance_value}
    return result_dict
    

In [33]:
result_dict = get_mean_var(train_data, data_dir)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 14681/14681 [01:57<00:00, 124.69it/s]


In [34]:
set(result_dict['CO2 (ETCO2, PCO2, etc.)'])

{'mean', 'variance'}

In [ ]:
len(result_dict)

In [ ]:
variable_values = {k : [] for k in df.columns[1:]}

In [ ]:
from tqdm import tqdm

for sample_path in tqdm(train_data['stay']):
    sample_path = data_dir+sample_path
    df = pd.read_csv(sample_path)
    df.drop(labels=categorical_variables, axis=1, inplace=True)
    cols = df.columns[1:]
    df = df[cols]
    values = df.values
    for i in range(values.shape[0]):
        for j in range(values.shape[1]):
            try :
                np.isnan(values[i][j])
            except:
                print(values[i][j])
            if np.isnan(values[i][j]) == False:
                variable_values[id_name_dict[j+1]].append(values[i][j])
    result_dict = {}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.distplot(variable_values['Capillary refill rate'], hist=True, bins = 100)

In [ ]:
result_dict = {}
for feature, values in variable_values.items():
    mean_value = np.mean(values)
    variance_value = np.var(values)
    result_dict[feature] = {'mean': mean_value, 'variance': variance_value}

In [ ]:
result_dict

In [ ]:
variable_values['pH']

In [ ]:
np.mean(variable_values['Capillary refill rate'])

In [ ]:
np.std(variable_values['Capillary refill rate'])

In [ ]:
np.max(variable_values['Capillary refill rate'])

In [ ]:
np.min(variable_values['Capillary refill rate'])

In [ ]:
class MimicDataSetInHospitalMortality(Dataset):
    def __init__(self, data_dir, csv_file, mean_variance, mode, seq_len, pad_value = 0, device = DEVICE):
        super().__init__() 
        self.data_dir = data_dir
        self.csv_file = csv_file
        self.seq_len = seq_len
        self.mode = mode
        self.data_df = pd.read_csv(csv_file)
        self.mean_variance = mean_variance
        self.pad_value = pad_value
        self.device = device
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        path = self.data_dir + self.data_df['stay'][idx]
        data = pd.read_csv(path)
        categorical_variables = ['Glascow coma scale eye opening', 
                                 'Glascow coma scale motor response', 
                                 'Glascow coma scale verbal response']
        id_name_dict = {}
        
        data.drop(labels=categorical_variables, axis=1, inplace=True)
        
        # for col in categorical_variables:
        #     print(data[col].unique())
        #     data[col] = data[col].apply(lambda x : float(x.split()[0]) if str(x)==x else x)

        for i in range(len(data.columns)):
            id_name_dict[i] = data.columns[i]
        values = data.values
        sample = self.extract(values, id_name_dict)
        # print(len(sample[0]))
        if len(sample[0]) >= self.seq_len :
            sample[0] = sample[0][-self.seq_len:]
            sample[1] = sample[1][-self.seq_len:]
            sample[2] = sample[2][-self.seq_len:]
            sample[3] = sample[3][-self.seq_len:]
        num_padd_tokens = self.seq_len - len(sample[0])
        
        variable_input = torch.cat([
            torch.tensor(sample[2], dtype=torch.int64),
            torch.tensor([self.pad_value]*num_padd_tokens, dtype=torch.int64)
        ])
        value_input = torch.cat([
            torch.tensor(sample[1], dtype=torch.float),
            torch.tensor([self.pad_value]*num_padd_tokens, dtype=torch.float)
        ])
        val = torch.tensor(sample[0], dtype=torch.float)
        # print(val)
        # print(val.min())
        time_input = torch.cat([
             val - val.min() ,
            torch.tensor([self.pad_value]*num_padd_tokens, dtype=torch.float)
        ])
        variables = sample[3] + ['pad token']*num_padd_tokens
        
        # print(variable_input.shape)
        # print(value_input.shape)
        # print(time_input.shape)
        # print(len(variables))
        assert variable_input.size(0) == self.seq_len
        assert value_input.size(0) == self.seq_len
        assert time_input.size(0) == self.seq_len
        
        return {
            "encoder_input" : [time_input.to(self.device), variable_input.to(self.device), value_input.to(self.device)],
            "encoder_mask": (variable_input != self.pad_value).unsqueeze(0).int().to(self.device),
            "variables" : variables,
            "label" : torch.tensor([self.data_df['y_true'][idx]], dtype=torch.int64).to(self.device)
        }
    
    def extract(self, values, id_name_dict):
        sample = [[],[],[],[]]
        start_time = values[0,0]
        for i in range(values.shape[0]):
            time = values[i,0]
            for j in range(1, values.shape[1]):
                try :
                    np.isnan(values[i][j])
                except:
                    print(values[i][j])
                if np.isnan(values[i][j]) == False:
                    sample[0].append(time)
                    mean = self.mean_variance[id_name_dict[j]]['mean']
                    std = self.mean_variance[id_name_dict[j]]['variance']**0.5
                    sample[1].append((values[i][j]-mean)/std)
                    sample[2].append(j)
                    sample[3].append(id_name_dict[j])
        return sample

In [ ]:
from tqdm import tqdm

In [ ]:
# lengths = []
# MAX_LEN = 0
# for data_path in tqdm(train_data['stay']):
#     path = data_dir+data_path
#     df = pd.read_csv(path)
#     l = (df.isnull() == False).sum().sum()
#     lengths.append((df.isnull() == False).sum().sum())
#     if l > MAX_LEN:
#         MAX_LEN = l

In [ ]:
# MAX_LEN

In [ ]:
MAX_LEN = 22048

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(lengths, bins='auto', alpha=0.7, color='blue', edgecolor='black')

# # Add labels and title
# plt.xlabel('Lengths')
# plt.ylabel('Frequency')
# plt.title('Distribution of Lengths')

# # Show the plot
# plt.show()

In [ ]:
MAX_LEN = 25 # donot change this
batch_size = 32
d_model = 32
num_heads = 4
N = 2
num_variables = 14 
num_variables+=1 #for no variable embedding while doing padding
d_ff = 64
epochs = 1
learning_rate = 3e-4
drop_out = 0.2

In [ ]:
train_data_path = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/train_listfile.csv"
val_data_path = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/val_listfile.csv"

data_dir = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/train/"

mean_variance = get_mean_var(pd.read_csv(train_data_path), data_dir)

train_ds = MimicDataSetInHospitalMortality(data_dir, train_data_path, mean_variance, 'training', MAX_LEN)
val_ds = MimicDataSetInHospitalMortality(data_dir, val_data_path, mean_variance, 'validation', MAX_LEN)

In [ ]:
train_dataloader = DataLoader(train_ds, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size = batch_size, shuffle=True)

In [ ]:
from tqdm import tqdm
for batch in tqdm(train_dataloader):
    pass

In [ ]:
batch['encoder_input'][0]

In [ ]:
batch['encoder_input'][0].min(axis=1).values.unsqueeze(1).shape

In [ ]:
batch['encoder_input'][0] - batch['encoder_input'][0].min(axis=1).values.unsqueeze(1)

In [ ]:
batch['encoder_input'][1]

In [ ]:
batch['encoder_input'][2]

In [ ]:
batch['encoder_mask'].shape

In [ ]:
batch['label'].shape

In [ ]:
class ContinuousValueEmbedding(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.embedding = nn.Linear(1, d_model)
        self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.embedding(x.unsqueeze(2))
        out = self.tanh(out)
        return out
    
class VariableEmbedding(nn.Module):
    def __init__(self, d_model, num_variables):
        super().__init__()
        self.embedding = nn.Embedding(num_variables+1, d_model)
        
    def forward(self, x):
        return self.embedding(x)

In [ ]:
cvs = ContinuousValueEmbedding(d_model).to(DEVICE)

In [ ]:
time_input = batch['encoder_input'][0]
variable_input = batch['encoder_input'][1]
value_input = batch['encoder_input'][2]

In [ ]:
value_input.shape

In [ ]:
cvs.forward(value_input).shape

In [ ]:
value_embedding = cvs.forward(value_input)

In [ ]:
cvs.forward(time_input).shape

In [ ]:
time_embedding = cvs.forward(time_input)

In [ ]:
time_embedding.shape

In [ ]:
varembed = VariableEmbedding(d_model, num_variables).to(DEVICE)

In [ ]:
varembed.forward(variable_input).shape

In [ ]:
variable_embedding = varembed.forward(variable_input)

In [ ]:
variable_embedding.shape

In [ ]:
class Embedding(nn.Module):
    def __init__(self, d_model, num_variables):
        super().__init__()
        self.cvs_value = ContinuousValueEmbedding(d_model)
        self.cvs_time = ContinuousValueEmbedding(d_model)
        self.var_embed = VariableEmbedding(d_model, num_variables)
    def forward(self, encoder_input):
        time = encoder_input[0]
        variable = encoder_input[1]
        value = encoder_input[2]
        embed = self.cvs_time(time) + self.cvs_value(value) + self.var_embed(variable)
        return embed

In [ ]:
embedding = Embedding(d_model, num_variables).to(DEVICE)

In [ ]:
embedding.forward(batch['encoder_input']).shape

In [ ]:
embed = embedding.forward((time_input, variable_input, value_input))

In [ ]:
embed.shape

In [ ]:
class Attention(nn.Module):
    def __init__(self, d_model, d, dropout=0.2):
        super().__init__()
        self.d_model = d_model
        self.d = d
        self.Q = nn.Linear(d_model, d)
        self.K = nn.Linear(d_model, d)
        self.V = nn.Linear(d_model, d)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x, mask): #inp --> (64, 256, d_model)
        q = self.Q(x) #(64, 256, d)
        k = self.K(x) #(64, 256, d)
        v = self.V(x) #(64, 256, d)
        # print(q.shape)
        weights = q@k.transpose(-2,-1)*k.shape[-1]**(-0.5) #(64, 256, 256) 
        # print(weights.shape)
        # weights = weights.masked_fill(torch.logical_or(mask == 0, (mask.transpose(-2, -1) == 0)), float('-inf'))
        weights = weights.masked_fill(mask == 0, float('-inf'))
        # print(weights)
        weights = F.softmax(weights, dim = -1) #(64, 256, 256)
        self.dropout(weights)
        out = weights @ v
        return out #(64, 256, d)

In [ ]:
attn = Attention(d_model, 8).to(DEVICE)

In [ ]:
attn_out = attn.forward(embed, batch['encoder_mask'])

In [ ]:
attn_out.shape

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout = 0.2):
        super().__init__()
        self.heads = nn.ModuleList([Attention(d_model, d_model//n_heads) for _ in range(n_heads)])
        self.proj = nn.Linear(n_heads*(d_model//n_heads), d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        out = torch.cat([h(x, mask) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
m_att = MultiHeadAttention(d_model, 4).to(DEVICE)

In [ ]:
matt_out = m_att(embed, batch['encoder_mask'])

In [ ]:
matt_out.shape

In [ ]:
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model, d_ff, dropout = 0.2):
        super().__init__()
        self.dropout = nn.Dropout(0.2)
        self.W1 = nn.Linear(d_model, d_ff)
        self.W2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        out = self.W1(x)
        out = F.relu(out)
        out = self.dropout(self.W2(out))
        return out

In [ ]:
ffb = FeedForwardBlock(d_model, d_ff).to(DEVICE)

In [ ]:
ffb(matt_out).shape

In [ ]:
ffb_out = ffb(matt_out)

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.multi_attention = MultiHeadAttention(d_model, n_heads)
        self.ffb = FeedForwardBlock(d_model, d_ff)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
    
    def forward(self, x, mask):
        out = self.multi_attention(x, mask)
        out1 = x + self.ln2(out)
        out2 = self.ffb(out1)
        out = out1 + self.ln2(out2)
        return out

In [ ]:
encoder_block = EncoderBlock(d_model, num_heads, d_ff).to(DEVICE)

In [ ]:
out = encoder_block(embed, batch['encoder_mask'])

In [ ]:
out.shape

In [ ]:
class Encoder(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, num_variables , N):
        super().__init__()
        self.embedding = Embedding(d_model, num_variables)
        self.encoder_blocks = nn.ModuleList([EncoderBlock(d_model, n_heads, d_ff) for _ in range(N)])
        self.N = N
    
    def forward(self, encoder_input, mask):
        time = encoder_input[0]
        variable = encoder_input[1]
        value = encoder_input[2]
        x = self.embedding((time, variable, value))
        for block in self.encoder_blocks:
            x = block(x, mask)
        return x

In [ ]:
encoder = Encoder(d_model, num_heads, d_ff, num_variables, N).to(DEVICE)

In [ ]:
enc_out = encoder.forward(embed, batch['encoder_mask'])

In [ ]:
class FusionSelfAttention(nn.Module):
    def __init__(self, d_model, dropout = 0.2):
        super().__init__()
        self.Wa = nn.Linear(d_model, d_model)
        self.Ua = nn.Linear(d_model, d_model)
        self.Va = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, out, mask):
        q = out.unsqueeze(2) #(4, 100, 1,  32)
        k = out.unsqueeze(1) #(4, 1, 100, 32)
        v = out #(4, 100, 32)
        a = F.tanh(self.Wa(q) + self.Ua(k)) #(4, 100, 100, 32)
        # print(a.shape)
        wei = self.Va(self.dropout(a)).squeeze()#(4, 100, 100)
        # print(wei.shape)
        wei = wei.masked_fill(mask == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        out = wei@v
        return out
        

In [ ]:
fsa = FusionSelfAttention(d_model).to(DEVICE)

In [ ]:
fsa(enc_out, batch['encoder_mask']).shape

In [ ]:
class Model(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, num_variables, N):
        super().__init__()
        self.encoder = Encoder(d_model, n_heads, d_ff, num_variables, N)
        self.fsa = FusionSelfAttention(d_model)
        self.proj = nn.Linear(d_model, 1)
    
    def forward(self, x, mask):
        out = self.encoder(x, mask)
        out = self.fsa(out, mask)
        out = out.masked_fill(mask.transpose(-2,-1)==0, 0)
        out = out.sum(dim = 1)
        out = self.proj(out)
        return out

In [ ]:
model = Model(d_model, num_heads, d_ff, num_variables, N).to(DEVICE)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

In [ ]:
model(batch['encoder_input'], batch['encoder_mask']).shape

In [ ]:
# from sklearn.metrics import roc_auc_score
# def eval():
#     model.eval()
#     all_labels

In [ ]:
from tqdm import tqdm
model = Model(d_model, num_heads, d_ff, num_variables, N).to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(epochs):
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False):
        outputs = model(batch['encoder_input'], batch['encoder_mask'])
        loss = criterion(outputs, batch['label'].float().view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

In [ ]:
from sklearn.metrics import roc_auc_score

def calculate_roc_auc(model, data_loader):
    model.eval()
    all_probabilities = []
    all_labels = []
    
    with torch.no_grad():
        for inputs in tqdm(data_loader, leave=False):
            outputs = model(inputs['encoder_input'], inputs['encoder_mask'])
            labels = inputs['label']
            logits = torch.sigmoid(outputs)
            
            all_probabilities.append(logits.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    logits_all = np.concatenate(all_probabilities)
    labels_all = np.concatenate(all_labels)
    
    roc_auc = roc_auc_score(labels_all, logits_all)
    return roc_auc

In [ ]:
calculate_roc_auc(model, val_dataloader)

In [ ]:
import pandas as pd
import numpy as np

file_path = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/train/listfile.csv"

files = pd.read_csv(file_path)

In [ ]:
files

In [ ]:
zero_indices = files[files['y_true'] == 0].index.values

In [ ]:
len(zero_indices)

In [ ]:
one_indices = files[files['y_true'] == 1].index.values

In [ ]:
len(one_indices)

In [ ]:
np.random.shuffle(zero_indices)
np.random.shuffle(one_indices)

In [ ]:
train_size = 0.8

In [ ]:
N = int(len(zero_indices)*train_size)
train_zero_indices = zero_indices[:N]
val_zero_indices = zero_indices[N:]

N = int(len(one_indices)*train_size)
train_one_indices = one_indices[:N]
val_one_indices = one_indices[N:]

In [ ]:
train_zero_data = files.iloc[train_zero_indices,:]
val_zero_data = files.iloc[val_zero_indices,:]
train_one_data = files.iloc[train_one_indices,:]
val_one_data = files.iloc[val_one_indices,:]

In [ ]:
train_list_file = pd.concat([train_one_data, train_zero_data]).values

In [ ]:
val_list_file = pd.concat([val_one_data, val_zero_data]).values

In [ ]:
train_indices = list(range(len(train_list_file)))
val_indices = list(range(len(val_list_file)))

np.random.shuffle(train_indices)
np.random.shuffle(val_indices)

train_list_file = train_list_file[train_indices][:]
val_list_file = val_list_file[val_indices][:]

In [ ]:
train_list_file

In [ ]:
val_list_file

In [ ]:
train_list_file.shape

In [ ]:
val_list_file.shape

In [ ]:
train_list_file = pd.DataFrame(train_list_file,columns=['stay', 'y_true'])
val_list_file = pd.DataFrame(val_list_file,columns=['stay', 'y_true'])

In [ ]:
train_list_file.to_csv(f"{data_dir}/train_listfile.csv", index = False)
val_list_file.to_csv(f"{data_dir}/val_list_file.csv", index = False)

In [ ]:
val_list_file.describe()

In [ ]:
MAX_LEN = 448
batch_size = 32
d_model = 50
num_heads = 4
N = 2
num_variables = 17 
num_variables += 1 #for no variable embedding while doing padding
d_ff = 100
epochs = 75
learning_rate = 8e-4
drop_out = 0.2
sinusoidal = True
import torch
DEVICE = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'

import pandas as pd
import numpy as np
from utils import MimicDataSetInHospitalMortality, calculate_roc_auc, calculate_auc_prc

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn import functional as F

from model import Model
from tqdm import tqdm
from normalizer import Normalizer
from categorizer import Categorizer

train_data_path = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/train_listfile.csv"
val_data_path = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/val_listfile.csv"

data_dir = "/data/datasets/mimic3-benchmarks/data/in-hospital-mortality/train/"

import pickle

with open('normalizer.pkl', 'rb') as file:
    normalizer = pickle.load(file)

with open('categorizer.pkl', 'rb') as file:
    categorizer = pickle.load(file)
    

mean_variance = normalizer.mean_var_dict
cat_dict = categorizer.category_dict


train_ds = MimicDataSetInHospitalMortality(data_dir, train_data_path, mean_variance, cat_dict, 'training', MAX_LEN)
val_ds = MimicDataSetInHospitalMortality(data_dir, val_data_path, mean_variance, cat_dict, 'validation', MAX_LEN)

train_dataloader = DataLoader(train_ds, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size = 1, shuffle=True)

model = Model(d_model, num_heads, d_ff, num_variables, N, sinusoidal).to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

for epoch in range(epochs):
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False):
        inp = batch['encoder_input']
        mask = batch['encoder_mask']
        y = batch['label']
        outputs = model(inp, mask)
        loss = criterion(outputs, y.float().view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # print(f'Epoch {epoch + 1}/{epochs}, Train AUC-ROC: {calculate_roc_auc(model, train_dataloader):.3f}')
    print(f'Epoch {epoch + 1}/{epochs}, Validation AUC-ROC: {calculate_roc_auc(model, val_dataloader):.3f}')
    print(f'Epoch {epoch + 1}/{epochs}, Validation AUC-PRC: {calculate_auc_prc(model, val_dataloader):.3f}')
    

# Constructing the file path
file_path = f"model_maxlen{MAX_LEN}_batch{batch_size}_dmodel{d_model}_heads{num_heads}_N{N}_vars{num_variables}_dff{d_ff}_epochs{epochs}_lr{learning_rate}_dropout{drop_out}_sinusoidal{sinusoidal}.pth"

# Example usage
torch.save(model.state_dict(), "models/"+ file_path)


In [ ]:
for batch in tqdm(train_dataloader):
    pass

In [ ]:
batch['encoder_input'][2].shape

In [ ]:
batch['encoder_input'][1].shape

In [ ]:
batch['encoder_input'][1].max()

In [ ]:
batch['encoder_input'][1].min()